Для выполнения домашнего задания необходимо взять boston house-prices datase (sklearn.datasets.load_boston) и сделать тоже самое для задачи регрессии (попробовать разные алгоритмы, поподбирать параметры, вывести итоговое качество). 

In [1]:
import warnings
warnings.filterwarnings("ignore") 
import pandas as pd
import numpy as np

from sklearn.preprocessing import StandardScaler

from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import SGDRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

from sklearn.metrics import r2_score

from sklearn import datasets

from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, RandomizedSearchCV

In [2]:
df = datasets.load_boston()

In [3]:
X = pd.DataFrame(data = df.data, columns = df.feature_names)
y = pd.Series(df.target)

In [4]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 506 entries, 0 to 505
Data columns (total 13 columns):
CRIM       506 non-null float64
ZN         506 non-null float64
INDUS      506 non-null float64
CHAS       506 non-null float64
NOX        506 non-null float64
RM         506 non-null float64
AGE        506 non-null float64
DIS        506 non-null float64
RAD        506 non-null float64
TAX        506 non-null float64
PTRATIO    506 non-null float64
B          506 non-null float64
LSTAT      506 non-null float64
dtypes: float64(13)
memory usage: 51.5 KB


In [5]:
X.describe()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT
count,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000
mean,3.613524,11.363636,11.136779,0.069170,0.554695,6.284634,68.574901,3.795043,9.549407,408.237154,18.455534,356.674032,12.653063
std,8.601545,23.322453,6.860353,0.253994,0.115878,0.702617,28.148861,2.105710,8.707259,168.537116,2.164946,91.294864,7.141062
min,0.006320,0.000000,0.460000,0.000000,0.385000,3.561000,2.900000,1.129600,1.000000,187.000000,12.600000,0.320000,1.730000
25%,0.082045,0.000000,5.190000,0.000000,0.449000,5.885500,45.025000,2.100175,4.000000,279.000000,17.400000,375.377500,6.950000
50%,0.256510,0.000000,9.690000,0.000000,0.538000,6.208500,77.500000,3.207450,5.000000,330.000000,19.050000,391.440000,11.360000
75%,3.677083,12.500000,18.100000,0.000000,0.624000,6.623500,94.075000,5.188425,24.000000,666.000000,20.200000,396.225000,16.955000
max,88.976200,100.000000,27.740000,1.000000,0.871000,8.780000,100.000000,12.126500,24.000000,711.000000,22.000000,396.900000,37.970000


In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 20)

In [7]:
scaler=StandardScaler()
scaler.fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_train_scaled = pd.DataFrame(X_train_scaled, columns=X_train.columns)

In [8]:
X_test_scaled = scaler.transform(X_test)
X_test_scaled = pd.DataFrame(X_test_scaled, columns=X_test.columns)

In [9]:
result = {}

# Линейная регрессия

In [10]:
model_linreg = LinearRegression()
model_linreg.fit(X_train_scaled, y_train)
y_pred = model_linreg.predict(X_test_scaled)
r2_score(y_test, y_pred)

0.7023964981707975

In [11]:
linreg_params=[{
    'fit_intercept': ['False', "True"],
    }]

In [12]:
linreg_gs = GridSearchCV(model_linreg, param_grid = linreg_params, cv=10, scoring='r2', return_train_score=True)
linreg_gs.fit(X_train_scaled, y_train)

/Users/lera/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


GridSearchCV(cv=10, error_score='raise-deprecating',
       estimator=LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid=[{'fit_intercept': ['False', 'True']}],
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring='r2', verbose=0)

In [13]:
linreg_gs.best_params_

{'fit_intercept': 'False'}

In [14]:
linreg_gs.best_score_

0.6817354004277203

In [15]:
model_linreg_gs=linreg_gs.best_estimator_

In [16]:
model_linreg_gs.fit(X_train_scaled, y_train)
y_pred = model_linreg_gs.predict(X_test_scaled)
r2_score(y_test, y_pred)

0.7023964981707975

# Лассо регрессия

In [17]:
model_lassoreg = Lasso()
model_lassoreg.fit(X_train_scaled, y_train)
y_pred = model_lassoreg.predict(X_test_scaled)
r2_score(y_test, y_pred)

0.5850666523531671

In [18]:
lasso_params = [{
    'alpha': (0.001, 0.01, 0.1, 0.5, 1, 1.5, 2, 5, 10)
}]

In [19]:
lasso_gs = GridSearchCV(model_lassoreg, param_grid = lasso_params, cv=10, scoring='r2', return_train_score=True)
lasso_gs.fit(X_train_scaled, y_train)

/Users/lera/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


GridSearchCV(cv=10, error_score='raise-deprecating',
       estimator=Lasso(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=1000,
   normalize=False, positive=False, precompute=False, random_state=None,
   selection='cyclic', tol=0.0001, warm_start=False),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid=[{'alpha': (0.001, 0.01, 0.1, 0.5, 1, 1.5, 2, 5, 10)}],
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring='r2', verbose=0)

In [20]:
lasso_gs.best_params_

{'alpha': 0.01}

In [21]:
lasso_gs.best_score_

0.6818296270679444

In [22]:
model_lasso_gs = lasso_gs.best_estimator_

In [23]:
model_lasso_gs.fit(X_train_scaled, y_train)
y_pred = model_lasso_gs.predict(X_test_scaled)
r2_score(y_test, y_pred)

0.7005626214712399

# SGDRegressor

In [24]:
model_SGDReg = SGDRegressor(random_state = 20)
model_SGDReg.fit(X_train_scaled, y_train)
y_pred = model_SGDReg.predict(X_test_scaled)
r2_score(y_test, y_pred)

0.6809245057684759

In [25]:
SGDReg_params = [{
    'alpha': (0.001, 0.01, 0.1, 0.5, 1, 1.5, 2, 5, 10),
    'epsilon': (0.001, 0.01, 0.1, 0.5, 1, 1.5, 2, 5, 10)
}]

In [26]:
SGDReg_gs = GridSearchCV(model_SGDReg, param_grid = SGDReg_params, cv=10, scoring='r2', return_train_score=True)
SGDReg_gs.fit(X_train_scaled, y_train)

/Users/lera/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


GridSearchCV(cv=10, error_score='raise-deprecating',
       estimator=SGDRegressor(alpha=0.0001, average=False, early_stopping=False, epsilon=0.1,
       eta0=0.01, fit_intercept=True, l1_ratio=0.15,
       learning_rate='invscaling', loss='squared_loss', max_iter=None,
       n_iter=None, n_iter_no_change=5, penalty='l2', power_t=0.25,
       random_state=20, shuffle=True, tol=None, validation_fraction=0.1,
       verbose=0, warm_start=False),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid=[{'alpha': (0.001, 0.01, 0.1, 0.5, 1, 1.5, 2, 5, 10), 'epsilon': (0.001, 0.01, 0.1, 0.5, 1, 1.5, 2, 5, 10)}],
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring='r2', verbose=0)

In [27]:
SGDReg_gs.best_params_

{'alpha': 0.1, 'epsilon': 0.001}

In [28]:
SGDReg_gs.best_score_

0.6740958329639983

In [29]:
model_SGDReg_gs = SGDReg_gs.best_estimator_

In [30]:
model_SGDReg_gs.fit(X_train_scaled, y_train)
y_pred = model_SGDReg_gs.predict(X_test_scaled)
r2_score(y_test, y_pred)

0.672384236526947

# KNeighborsRegressor

In [31]:
model_KNeighborsReg = KNeighborsRegressor()
model_KNeighborsReg.fit(X_train_scaled, y_train)
y_pred = model_KNeighborsReg.predict(X_test_scaled)
r2_score(y_test, y_pred)

0.7388194206030703

In [32]:
KNeighborsReg_params = [{
    'n_neighbors': (range(3, 31)),    
}]

In [33]:
KNeighborsReg_gs = GridSearchCV(model_KNeighborsReg, param_grid = KNeighborsReg_params, cv=10, scoring='r2', return_train_score=True)
KNeighborsReg_gs.fit(X_train_scaled, y_train)

/Users/lera/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


GridSearchCV(cv=10, error_score='raise-deprecating',
       estimator=KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
          metric_params=None, n_jobs=None, n_neighbors=5, p=2,
          weights='uniform'),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid=[{'n_neighbors': range(3, 31)}], pre_dispatch='2*n_jobs',
       refit=True, return_train_score=True, scoring='r2', verbose=0)

In [34]:
KNeighborsReg_gs.best_params_

{'n_neighbors': 3}

In [35]:
KNeighborsReg_gs.best_score_

0.7415126757141723

In [36]:
model_KNeighborsReg_gs = KNeighborsReg_gs.best_estimator_

In [37]:
model_KNeighborsReg_gs.fit(X_train_scaled, y_train)
y_pred = model_KNeighborsReg_gs.predict(X_test_scaled)
r2_score(y_test, y_pred)

0.773404568608788

# DecisionTreeRegressor

In [38]:
model_TreeReg = DecisionTreeRegressor(random_state=20)
model_TreeReg.fit(X_train_scaled, y_train)
y_pred = model_TreeReg.predict(X_test_scaled)
r2_score(y_test, y_pred)

0.6451102498186433

In [39]:
tree_params=[{
    'max_features': [1, 0.8, 0.6, 0.4, 0.2, 0.1],
    'max_depth': list(range(1,17,3)),
    'min_samples_leaf': list(range(1,10,2)),
}]

In [40]:
TreeReg_gs = GridSearchCV(model_TreeReg, param_grid = tree_params, cv=10, scoring='r2', return_train_score=True)
TreeReg_gs.fit(X_train_scaled, y_train)

/Users/lera/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


GridSearchCV(cv=10, error_score='raise-deprecating',
       estimator=DecisionTreeRegressor(criterion='mse', max_depth=None, max_features=None,
           max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           presort=False, random_state=20, splitter='best'),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid=[{'max_features': [1, 0.8, 0.6, 0.4, 0.2, 0.1], 'max_depth': [1, 4, 7, 10, 13, 16], 'min_samples_leaf': [1, 3, 5, 7, 9]}],
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring='r2', verbose=0)

In [41]:
TreeReg_gs.best_params_

{'max_depth': 10, 'max_features': 0.4, 'min_samples_leaf': 3}

In [42]:
TreeReg_gs.best_score_

0.8082224040507899

In [43]:
model_TreeReg_gs = TreeReg_gs.best_estimator_

In [44]:
model_TreeReg_gs.fit(X_train_scaled, y_train)
y_pred = model_TreeReg_gs.predict(X_test_scaled)
r2_score(y_test, y_pred)

0.6950398245901945

# RandomForestRegressor

In [45]:
model_RFReg = RandomForestRegressor(random_state=20)
model_RFReg.fit(X_train_scaled, y_train)
y_pred = model_TreeReg.predict(X_test_scaled)
r2_score(y_test, y_pred)

0.6451102498186433

In [46]:
rf_params=[{
    'n_estimators': [100, 80, 50, 30, 20, 10],
    'max_depth': list(range(1,17,3)),
    'min_samples_leaf': list(range(1,7,2)),
}]

In [47]:
RFReg_gs = GridSearchCV(model_RFReg, param_grid = rf_params, cv=10, scoring='r2', return_train_score=True)
RFReg_gs.fit(X_train_scaled, y_train)

/Users/lera/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


GridSearchCV(cv=10, error_score='raise-deprecating',
       estimator=RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
           oob_score=False, random_state=20, verbose=0, warm_start=False),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid=[{'n_estimators': [100, 80, 50, 30, 20, 10], 'max_depth': [1, 4, 7, 10, 13, 16], 'min_samples_leaf': [1, 3, 5]}],
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring='r2', verbose=0)

In [48]:
RFReg_gs.best_params_

{'max_depth': 7, 'min_samples_leaf': 1, 'n_estimators': 50}

In [49]:
RFReg_gs.best_score_

0.8313997452885614

In [50]:
model_RFReg_gs = RFReg_gs.best_estimator_

In [51]:
model_RFReg_gs.fit(X_train_scaled, y_train)
y_pred = model_RFReg_gs.predict(X_test_scaled)
r2_score(y_test, y_pred)

0.6561661502380303